In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from geopy.geocoders import Nominatim
import warnings
import requests
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv('/Users/munjismac/code/munjik/personal-projects/real_estate_project/RawData.csv')

In [3]:
df.head(10)

,CASE #,Special Master,Lender,Debtor,Additional Lien,Date,Time,Address,City,State,Zip,Balance,ARV,EQUITY,Sold,Postponed
0,D-202-CV-2019-00076,Robert Doyle,Loandepot.com,"Altamirano, Dominic; Altamirano, Julia",Cavalry SPV I; NMTR,1/5/21,10:15 AM,10528 Duke Ave SW,Albuquerque,NM,87121,"$97,318.40","$145,000.00","$47,681.60",NaN,CANCELLED
1,D-202-CV-2018-03849,Robert Doyle,US Bank,"Conlee, Nicole",HUD,1/5/21,10:15 AM,4605 Palo Duro Ave NE,Albuquerque,NM,87110,"$95,564.54","$200,700.00","$105,135.46",NaN,3/2/21
2,D-202-CV-2016-07895,Robert Doyle,Pingora Loan Servicing,"Estate of Gallegos, Steven G; Gallegos, Doreen...",Time Investment Co,1/5/21,10:15 AM,2636 Don Felipe SW,Albuquerque,NM,87105,"$410,199.42","$361,000.00","($49,199.42)",273163,NaN
3,D-202-CV-2016-04200,Robert Doyle,US Bank,"Estate of Ulibarri, Gladys ; Ulibarri, Joseph ...",NaN,1/5/21,10:15 AM,7601 Winter Ave NE,Albuquerque,NM,87110,"$336,375.47","$307,000.00","($29,375.47)",NaN,2/2/21
4,D-202-CV-2018-04165,Robert Doyle,US Bank,"Garcia, Jamie S; Garcia, Miramon B",NMMFA,1/5/21,10:15 AM,413 Dolores Dr SW,Albuquerque,NM,87121,"$118,588.31","$149,000.00","$30,411.69",117450,NaN
5,D-202-CV-2015-04764,Robert Doyle,US Bank,"Giles, Larry G",NaN,1/5/21,10:15 AM,8123 Gem Pointe Rd SW,Albuquerque,NM,87121,"$168,188.76","$185,000.00","$16,811.24",180380.1,NaN
6,D-202-CV-2019-05980,Robert Doyle,"BOKF, NA","Kang, Joo E; Kang, Helen",NaN,1/5/21,10:15 AM,5008 General Bradley St NE,Albuquerque,NM,87111,"$270,255.41","$311,310.00","$41,054.59",NaN,3/2/21
7,D-202-CV-2017-02886,Robert Doyle,"BOKF, NA","Martinez, Betty J; Everly, Rebecca",NMTR,1/5/21,10:15 AM,2101 Spring Canyon Dr SW,Albuquerque,NM,87121,"$85,326.64","$175,000.00","$89,673.36",NaN,2/2/21
8,D-202-CV-2018-03958,Robert Doyle,US Bank,"Price, Timothy A",HUD; NMMFA; NMTR; MVD; LVNV Funding,1/5/21,10:15 AM,227 Hwy 217,Tijeras,NM,87059,"$120,141.43",NaN,NaN,NaN,NaN
9,D-202-CV-2018-05154,Robert Doyle,US Bank,"Smith, Doretha A",Aqua Finance; Citibank SD; NMTR,1/5/21,10:15 AM,9309 Halyard Rd NW,Albuquerque,NM,87121,"$171,443.30","$178,000.00","$6,556.70",108500,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CASE #           126 non-null    object
 1   Special Master   128 non-null    object
 2   Lender           128 non-null    object
 3   Debtor           128 non-null    object
 4   Additional Lien  96 non-null     object
 5   Date             128 non-null    object
 6   Time             128 non-null    object
 7   Address          128 non-null    object
 8   City             128 non-null    object
 9   State            128 non-null    object
 10  Zip              128 non-null    object
 11  Balance          128 non-null    object
 12  ARV              124 non-null    object
 13  EQUITY           123 non-null    object
 14  Sold             36 non-null     object
 15  Postponed        79 non-null     object
dtypes: object(16)
memory usage: 16.2+ KB


<h2>Let's clean up some data </h2> 

In [5]:
# remove the unwanted columns
df = df.drop(columns=['CASE #', 'Special Master','Additional Lien','Postponed', 'Time', 'Postponed','Debtor'])

In [6]:
df = df.drop(labels=128) #last column was all empty anyways
df.isnull().sum() #check to see how many null values we have in each column

Lender      0
Date        0
Address     0
City        0
State       0
Zip         0
Balance     0
ARV         4
EQUITY      5
Sold       92
dtype: int64

In [7]:
imputer = SimpleImputer(strategy="most_frequent") # Instanciate a SimpleImputer object with strategy of choice
# impute lender
imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = imputer.transform(df[['ARV']]) # Call the "transform" method on the object

imputer.fit(df[['EQUITY']]) # Call the "fit" method on the object
df['EQUITY'] = imputer.transform(df[['EQUITY']]) # Call the "transform" method on the object


In [8]:
df.head(10)

,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold
0,Loandepot.com,1/5/21,10528 Duke Ave SW,Albuquerque,NM,87121,"$97,318.40","$145,000.00","$47,681.60",NaN
1,US Bank,1/5/21,4605 Palo Duro Ave NE,Albuquerque,NM,87110,"$95,564.54","$200,700.00","$105,135.46",NaN
2,Pingora Loan Servicing,1/5/21,2636 Don Felipe SW,Albuquerque,NM,87105,"$410,199.42","$361,000.00","($49,199.42)",273163
3,US Bank,1/5/21,7601 Winter Ave NE,Albuquerque,NM,87110,"$336,375.47","$307,000.00","($29,375.47)",NaN
4,US Bank,1/5/21,413 Dolores Dr SW,Albuquerque,NM,87121,"$118,588.31","$149,000.00","$30,411.69",117450
5,US Bank,1/5/21,8123 Gem Pointe Rd SW,Albuquerque,NM,87121,"$168,188.76","$185,000.00","$16,811.24",180380.1
6,"BOKF, NA",1/5/21,5008 General Bradley St NE,Albuquerque,NM,87111,"$270,255.41","$311,310.00","$41,054.59",NaN
7,"BOKF, NA",1/5/21,2101 Spring Canyon Dr SW,Albuquerque,NM,87121,"$85,326.64","$175,000.00","$89,673.36",NaN
8,US Bank,1/5/21,227 Hwy 217,Tijeras,NM,87059,"$120,141.43","$190,000.00","$117,907.81",NaN
9,US Bank,1/5/21,9309 Halyard Rd NW,Albuquerque,NM,87121,"$171,443.30","$178,000.00","$6,556.70",108500


In [9]:
df.isnull().sum()

Lender      0
Date        0
Address     0
City        0
State       0
Zip         0
Balance     0
ARV         0
EQUITY      0
Sold       92
dtype: int64

In [10]:
# function to remove puncuation
def remove_dollar(x):
    return x.replace('$','')
def replace_comma(x):
    return x.replace(',','')
def replace_period(x):
    return x.replace('.','')
def replace_paran(x):
    return x.replace("(","").replace(")","")
# reformat from str to float
def currency_to_float(x: str) -> float:
    """Converts US currency ``x`` to float."""
    return float(x)
    
#-> potentially use this function to do all in one functions here
# Convert the currency to float.
# def currency_to_float(x: str) -> float:
#     """Converts US currency ``x`` to float."""
#     return float(x.replace("$", "").replace(",", ""))

In [11]:
df['Sold'] = df['Sold'].astype('str')
df['Sold'] = df['Sold'].apply(remove_dollar)
df['Sold'] = df['Sold'].apply(replace_comma)

In [12]:
df[(df.Sold) == 'THEY DONT HAVE ANYTHING ON THIS...he\'ll double check when back at office']
df.iloc[63]['Sold'] = 'NaN'
df['Sold'] = df['Sold'].apply(lambda x: currency_to_float(x))


In [13]:
# sum should have increased from 92 to 93
df['Sold'].isnull().sum()

93

In [14]:
mean_imputer = SimpleImputer(strategy="mean") # Instanciate a SimpleImputer object with strategy of choice
mean_imputer.fit(df[['Sold']]) # Call the "fit" method on the object
df['Sold'] = mean_imputer.transform(df[['Sold']]) # Call the "transform" method on the object

In [15]:
df['Sold'].isnull().sum() #now should be zero once we fill the values in with imputes

0

In [16]:
# First convert to type string -> clean Balance column
df['Balance'] = df['Balance'].astype('str')
df['Balance'] = df['Balance'].apply(replace_comma)
df['Balance'] = df['Balance'].apply(remove_dollar)

# First convert to type string -> clean ARV column
df['ARV'] = df['ARV'].astype('str')
df['ARV'] = df['ARV'].apply(replace_comma)
df['ARV'] = df['ARV'].apply(remove_dollar)
# First convert to type string -> clean Equity column
df['EQUITY'] = df['EQUITY'].astype('str')
df['EQUITY'] = df['EQUITY'].apply(replace_comma)
df['EQUITY'] = df['EQUITY'].apply(remove_dollar)
df['EQUITY'] = df['EQUITY'].apply(replace_paran)

In [17]:
# FIXING THIS ERROR: ValueError: could not convert string to float: 'COMMERCIAL'
df[(df.ARV) == 'COMMERCIAL'] #find location
df.loc[df.ARV == "COMMERCIAL", "ARV"] = None

In [18]:
df.isnull().sum() #sum for ARV has gone up to 1 because of code above
mean_imputer.fit(df[['ARV']]) # Call the "fit" method on the object
df['ARV'] = mean_imputer.transform(df[['ARV']]) # Call the "transform" method on the object

In [19]:
mean_imputer.statistics_

array([221713.1496063])

In [20]:
df['Balance'] = df['Balance'].apply(lambda x: currency_to_float(x))
df['ARV'] = df['ARV'].apply(lambda x: currency_to_float(x))
df['EQUITY'] = df['EQUITY'].apply(lambda x: currency_to_float(x))

In [21]:
# lets convert Date to datetime64
df['Date'] = pd.to_datetime(df['Date'])

In [22]:
df

,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold
0,Loandepot.com,2021-01-05,10528 Duke Ave SW,Albuquerque,NM,87121,97318.40,145000.0,47681.60,158512.216286
1,US Bank,2021-01-05,4605 Palo Duro Ave NE,Albuquerque,NM,87110,95564.54,200700.0,105135.46,158512.216286
2,Pingora Loan Servicing,2021-01-05,2636 Don Felipe SW,Albuquerque,NM,87105,410199.42,361000.0,49199.42,273163.000000
3,US Bank,2021-01-05,7601 Winter Ave NE,Albuquerque,NM,87110,336375.47,307000.0,29375.47,158512.216286
4,US Bank,2021-01-05,413 Dolores Dr SW,Albuquerque,NM,87121,118588.31,149000.0,30411.69,117450.000000
...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,247 Espanola NE,Albuquerque,NM,87108,81796.17,136000.0,54203.83,158512.216286
124,Deutsche Bank,2021-03-25,627 Cebolleta Ct SW,Albuquerque,NM,87105,160369.83,175000.0,14630.17,158512.216286
125,Rushmore Loan Management Services,2021-03-26,117 Sedilo Rd,Tijeras,NM,87059,189679.29,233000.0,43320.71,187301.000000
126,Freedom Mortgage Corp,2021-03-26,1301 Summerfield Pl SW,Albuquerque,NM,87121,154231.50,180000.0,25768.50,170600.000000


In [23]:
# function to concatante all coordinates into 1 for lats and longs in next step
def coordinates(dataframe_coord):
    cols = ["Address", "City","State","Zip"]
    df['coordinates'] = df[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    return df

In [24]:
coordinates(df)

,Lender,Date,Address,City,State,Zip,Balance,ARV,EQUITY,Sold,coordinates
0,Loandepot.com,2021-01-05,10528 Duke Ave SW,Albuquerque,NM,87121,97318.40,145000.0,47681.60,158512.216286,"10528 Duke Ave SW,Albuquerque,NM,87121"
1,US Bank,2021-01-05,4605 Palo Duro Ave NE,Albuquerque,NM,87110,95564.54,200700.0,105135.46,158512.216286,"4605 Palo Duro Ave NE,Albuquerque,NM,87110"
2,Pingora Loan Servicing,2021-01-05,2636 Don Felipe SW,Albuquerque,NM,87105,410199.42,361000.0,49199.42,273163.000000,"2636 Don Felipe SW,Albuquerque,NM,87105"
3,US Bank,2021-01-05,7601 Winter Ave NE,Albuquerque,NM,87110,336375.47,307000.0,29375.47,158512.216286,"7601 Winter Ave NE,Albuquerque,NM,87110"
4,US Bank,2021-01-05,413 Dolores Dr SW,Albuquerque,NM,87121,118588.31,149000.0,30411.69,117450.000000,"413 Dolores Dr SW,Albuquerque,NM,87121"
...,...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,247 Espanola NE,Albuquerque,NM,87108,81796.17,136000.0,54203.83,158512.216286,"247 Espanola NE,Albuquerque,NM,87108"
124,Deutsche Bank,2021-03-25,627 Cebolleta Ct SW,Albuquerque,NM,87105,160369.83,175000.0,14630.17,158512.216286,"627 Cebolleta Ct SW,Albuquerque,NM,87105"
125,Rushmore Loan Management Services,2021-03-26,117 Sedilo Rd,Tijeras,NM,87059,189679.29,233000.0,43320.71,187301.000000,"117 Sedilo Rd,Tijeras,NM,87059"
126,Freedom Mortgage Corp,2021-03-26,1301 Summerfield Pl SW,Albuquerque,NM,87121,154231.50,180000.0,25768.50,170600.000000,"1301 Summerfield Pl SW,Albuquerque,NM,87121"


In [28]:
#Turn our new coordinates column into Lats and Longs


def geolocate(dataframe):
    df_address = dataframe['coordinates']
    lats = []
    lngs = []
    #loop through or dataframe addresses
    for address in df_address:
        API_KEY = 'GGTqpEquAqOeHelGwevtJopD2QhYtUKl'
        base_url = 'http://www.mapquestapi.com/geocoding/v1/address'
        location = address

        #call the api function on all of our addresses
        params = {
            'key': API_KEY,
            'location' : location
        }

        response = requests.get(base_url,params=params).json()
        results = response['results'][0]
        lat = results['locations'][0]['latLng']['lat']
        lng = results['locations'][0]['latLng']['lng']
        lats.append(lat)
        lngs.append(lng)
    df_location = pd.DataFrame(list(zip(lats,lngs)),
                           columns=['location_lats', 'location_lngs'])
    dataframe = pd.concat([dataframe,df_location], axis=1)
    return dataframe
    

In [31]:
df = geolocate(df)

In [46]:
#dropping this column with useless info
df = df.drop(df.index[46])

In [59]:
df

,Lender,Date,City,State,Zip,Balance,ARV,EQUITY,Sold,location_lats,location_lngs
0,Loandepot.com,2021-01-05,Albuquerque,NM,87121,97318.40,145000.0,47681.60,158512.216286,35.065706,-106.748628
1,US Bank,2021-01-05,Albuquerque,NM,87110,95564.54,200700.0,105135.46,158512.216286,35.124107,-106.592768
2,Pingora Loan Servicing,2021-01-05,Albuquerque,NM,87105,410199.42,361000.0,49199.42,273163.000000,34.994331,-106.711337
3,US Bank,2021-01-05,Albuquerque,NM,87110,336375.47,307000.0,29375.47,158512.216286,35.094028,-106.562971
4,US Bank,2021-01-05,Albuquerque,NM,87121,118588.31,149000.0,30411.69,117450.000000,35.078276,-106.700572
...,...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,Albuquerque,NM,87108,81796.17,136000.0,54203.83,158512.216286,35.077974,-106.564131
124,Deutsche Bank,2021-03-25,Albuquerque,NM,87105,160369.83,175000.0,14630.17,158512.216286,35.076087,-106.695141
125,Rushmore Loan Management Services,2021-03-26,Tijeras,NM,87059,189679.29,233000.0,43320.71,187301.000000,35.109195,-106.310742
126,Freedom Mortgage Corp,2021-03-26,Albuquerque,NM,87121,154231.50,180000.0,25768.50,170600.000000,35.052685,-106.722726


In [ ]:
"""FOR REFERENCE ->
One Hot Encoding:
We use this categorical data encoding technique when the features are nominal(do not have any order)
------------------------------------------------------------------------------------------------
Label Encoding or Ordinal Encoding:
We use this categorical data encoding technique when the categorical feature is ordinal. In this case,
retaining the order is important. Hence encoding should reflect the sequence.
"""

In [50]:
print(df.Lender.nunique()) #OHE this or see next best step for 33 diff lenders
print(df.Address.nunique()) #--> I'd like to change this to Lat's and Longs for ML model
print(df.City.nunique()) #Encode this ? I think
print(df.State.nunique()) #easy encode all 1's
print(df.Zip.nunique())

32
90
2
1
16


In [51]:
#lets drop address and coordinates since we now have lats and longs
df = df.drop(columns=['Address', 'coordinates'])

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 127
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Lender         126 non-null    object        
 1   Date           126 non-null    datetime64[ns]
 2   City           126 non-null    object        
 3   State          126 non-null    object        
 4   Zip            126 non-null    object        
 5   Balance        126 non-null    float64       
 6   ARV            126 non-null    float64       
 7   EQUITY         126 non-null    float64       
 8   Sold           126 non-null    float64       
 9   location_lats  126 non-null    float64       
 10  location_lngs  126 non-null    float64       
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 11.8+ KB


In [56]:
#Change Zip to INT64 instead of current object
df['Zip'] = df['Zip'].astype('int64')

In [68]:
#begin OHE
ohc = OneHotEncoder()
ohe = ohc.fit_transform(df.Lender.values.reshape(-1,1)).toarray()

dfOneHot = pd.DataFrame(ohe, columns=['Lender_' +str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])
dfh = pd.concat([df, dfOneHot], axis = 1)

In [85]:
dfOneHot

,Lender_Alamitos Associates Management,"Lender_BOKF, NA",Lender_Bank of NY Mellon,Lender_Citibank,Lender_Citimortgage,Lender_Deutsche Bank,Lender_Freedom Mortgage Corp,Lender_Idaho Housing and Finance,Lender_LSF11 Master Participation Trust,Lender_LSF9 Master Participation Trust,...,Lender_Sandia Lab FCU,Lender_Seattle Bank,Lender_Specialized Loan Servicing,Lender_The Money Source,Lender_US Bank,Lender_US Eagle FCU,Lender_US NM FCU,Lender_Wells Fargo Bank,Lender_Wilmington Savings Fund Society,Lender_Wilmington Trust
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
dfh

,Lender,Date,City,State,Zip,Balance,ARV,EQUITY,Sold,location_lats,...,Lender_Sandia Lab FCU,Lender_Seattle Bank,Lender_Specialized Loan Servicing,Lender_The Money Source,Lender_US Bank,Lender_US Eagle FCU,Lender_US NM FCU,Lender_Wells Fargo Bank,Lender_Wilmington Savings Fund Society,Lender_Wilmington Trust
0,Loandepot.com,2021-01-05,Albuquerque,NM,87121.0,97318.40,145000.0,47681.60,158512.216286,35.065706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,US Bank,2021-01-05,Albuquerque,NM,87110.0,95564.54,200700.0,105135.46,158512.216286,35.124107,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Pingora Loan Servicing,2021-01-05,Albuquerque,NM,87105.0,410199.42,361000.0,49199.42,273163.000000,34.994331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,US Bank,2021-01-05,Albuquerque,NM,87110.0,336375.47,307000.0,29375.47,158512.216286,35.094028,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,US Bank,2021-01-05,Albuquerque,NM,87121.0,118588.31,149000.0,30411.69,117450.000000,35.078276,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,Albuquerque,NM,87108.0,81796.17,136000.0,54203.83,158512.216286,35.077974,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,Deutsche Bank,2021-03-25,Albuquerque,NM,87105.0,160369.83,175000.0,14630.17,158512.216286,35.076087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,Rushmore Loan Management Services,2021-03-26,Tijeras,NM,87059.0,189679.29,233000.0,43320.71,187301.000000,35.109195,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
126,Freedom Mortgage Corp,2021-03-26,Albuquerque,NM,87121.0,154231.50,180000.0,25768.50,170600.000000,35.052685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# starting off by starting to see what is selling above judgement and %%% sold vs Scheduled 

In [ ]:
#1.)if sold is above balanced 
#2.) change the objects to their types
#3.) NaN == 0
#4.) Banks to individual #'s' 1 - 33 top 10 banks
# data[(data.Medal == 'Silver') & (data.Gender == 'Men')]



In [86]:
#FOR TESTING PURPOSES ONLY
df

,Lender,Date,City,State,Zip,Balance,ARV,EQUITY,Sold,location_lats,location_lngs
0,Loandepot.com,2021-01-05,Albuquerque,NM,87121,97318.40,145000.0,47681.60,158512.216286,35.065706,-106.748628
1,US Bank,2021-01-05,Albuquerque,NM,87110,95564.54,200700.0,105135.46,158512.216286,35.124107,-106.592768
2,Pingora Loan Servicing,2021-01-05,Albuquerque,NM,87105,410199.42,361000.0,49199.42,273163.000000,34.994331,-106.711337
3,US Bank,2021-01-05,Albuquerque,NM,87110,336375.47,307000.0,29375.47,158512.216286,35.094028,-106.562971
4,US Bank,2021-01-05,Albuquerque,NM,87121,118588.31,149000.0,30411.69,117450.000000,35.078276,-106.700572
...,...,...,...,...,...,...,...,...,...,...,...
123,Wells Fargo Bank,2021-03-23,Albuquerque,NM,87108,81796.17,136000.0,54203.83,158512.216286,35.077974,-106.564131
124,Deutsche Bank,2021-03-25,Albuquerque,NM,87105,160369.83,175000.0,14630.17,158512.216286,35.076087,-106.695141
125,Rushmore Loan Management Services,2021-03-26,Tijeras,NM,87059,189679.29,233000.0,43320.71,187301.000000,35.109195,-106.310742
126,Freedom Mortgage Corp,2021-03-26,Albuquerque,NM,87121,154231.50,180000.0,25768.50,170600.000000,35.052685,-106.722726


In [87]:
testing_dataframe = df.drop(columns=['Lender'])

In [92]:
#extract month feature
months = df.Date.dt.month
#extract day of month feature
day_of_months = df.Date.dt.day

In [96]:
#make a dataframe with months and day of months 
data_feature = pd.DataFrame({
    'months':months,
    'days': day_of_months
})

In [99]:
feature = pd.concat([testing_dataframe, data_feature], axis=1)
feature = feature.drop(columns='Date')

In [162]:
#OHE city, NM, and ZIP.
feature

,City,State,Zip,Balance,ARV,EQUITY,Sold,location_lats,location_lngs,months,days
0,1.0,NM,87121,97318.40,145000.0,47681.60,158512.216286,35.065706,-106.748628,1,5
1,1.0,NM,87110,95564.54,200700.0,105135.46,158512.216286,35.124107,-106.592768,1,5
2,1.0,NM,87105,410199.42,361000.0,49199.42,273163.000000,34.994331,-106.711337,1,5
3,1.0,NM,87110,336375.47,307000.0,29375.47,158512.216286,35.094028,-106.562971,1,5
4,1.0,NM,87121,118588.31,149000.0,30411.69,117450.000000,35.078276,-106.700572,1,5
...,...,...,...,...,...,...,...,...,...,...,...
123,1.0,NM,87108,81796.17,136000.0,54203.83,158512.216286,35.077974,-106.564131,3,23
124,1.0,NM,87105,160369.83,175000.0,14630.17,158512.216286,35.076087,-106.695141,3,25
125,0.0,NM,87059,189679.29,233000.0,43320.71,187301.000000,35.109195,-106.310742,3,26
126,1.0,NM,87121,154231.50,180000.0,25768.50,170600.000000,35.052685,-106.722726,3,26


In [167]:
ohe = OneHotEncoder()
ohe.fit(feature[['State']]) # Fit encoder
feature['State'] = ohe.transform(feature[['State']]).toarray() # Encode Street


In [168]:
#use BinaryEncoder for ZIP

,City,State,Zip,Balance,ARV,EQUITY,Sold,location_lats,location_lngs,months,days
0,1.0,1.0,87121,97318.40,145000.0,47681.60,158512.216286,35.065706,-106.748628,1,5
1,1.0,1.0,87110,95564.54,200700.0,105135.46,158512.216286,35.124107,-106.592768,1,5
2,1.0,1.0,87105,410199.42,361000.0,49199.42,273163.000000,34.994331,-106.711337,1,5
3,1.0,1.0,87110,336375.47,307000.0,29375.47,158512.216286,35.094028,-106.562971,1,5
4,1.0,1.0,87121,118588.31,149000.0,30411.69,117450.000000,35.078276,-106.700572,1,5
...,...,...,...,...,...,...,...,...,...,...,...
123,1.0,1.0,87108,81796.17,136000.0,54203.83,158512.216286,35.077974,-106.564131,3,23
124,1.0,1.0,87105,160369.83,175000.0,14630.17,158512.216286,35.076087,-106.695141,3,25
125,0.0,1.0,87059,189679.29,233000.0,43320.71,187301.000000,35.109195,-106.310742,3,26
126,1.0,1.0,87121,154231.50,180000.0,25768.50,170600.000000,35.052685,-106.722726,3,26


<h2>Will work on this once basic model idea is done.. for now still quite of feature engineering to be done </h2>

In [149]:
ohc = OneHotEncoder()
final_ohe = ohc.fit_transform(df.City.values.reshape(-1,1)).toarray()
final_dfOneHot = pd.DataFrame(final_ohe, columns=['City_'+str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])

In [161]:
final_dfOneHot

,City_Albuquerque,City_Tijeras
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
121,1.0,0.0
122,1.0,0.0
123,0.0,1.0
124,1.0,0.0
